In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, TNK test function

This is the class method for running Xopt. Use Bayesian exploration to explore the input space.

TNK function
$n=2$ variables:
$x_i \in [0, \pi], i=1,2$

Objectives:
- $f_i(x) = x_i$

Constraints:
- $g_1(x) = -x_1^2 -x_2^2 + 1 + 0.1 \cos\left(16 \arctan \frac{x_1}{x_2}\right) \le 0$
- $g_2(x) = (x_1 - 1/2)^2 + (x_2-1/2)^2 \le 0.5$

In [2]:
# Import the class
from xopt import Xopt
from xopt.bayesian.generators.multi_fidelity import MultiFidelityGenerator
from xopt.bayesian.models.models import create_multi_fidelity_model
from botorch.test_functions.multi_fidelity import AugmentedHartmann


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
import yaml
# Make a proper input file. 
YAML = """
xopt: {output_path: null, verbose: true}

algorithm:
  name: multi_fidelity
  options:  
      n_initial_samples: 16
      n_steps: 10
      verbose: True
      use_gpu: False
      generator_options:                     ## options for bayesian exploration acquisition function
          batch_size: 4                      ## batch size for parallelized optimization


simulation: 
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = yaml.safe_load(YAML)

In [4]:
X = Xopt(config)
X

Loading config as dict.



            Xopt 
________________________________           
Version: 0.4.3+86.g7e21749.dirty
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: multi_fidelity
  function: xopt.bayesian.algorithms.multi_fidelity_optimize
  options:
    n_initial_samples: 16
    n_steps: 10
    verbose: true
    use_gpu: false
    generator_options: {batch_size: 4}
    custom_model: !!python/name:xopt.bayesian.models.models.create_multi_fidelity_model ''
    restart_file: null
    initial_x: null
simulation:
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate
  options: {extra_option: abc}
vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: {}
  constants: {a: dummy_constant}
 

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

executor = PoolExecutor()
# This will also work. 
#executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-08-19T11:39:41-05:00
started running optimization with generator: <xopt.bayesian.generators.multi_fidelity.MultiFidelityGenerator object at 0x00000219B9145F10>
submitting initial candidates at time 2021-08-19T11:39:41-05:00
starting optimization loop
Model creation time: 0.137 s
Candidate generation time: 26.16 s
Candidate(s): tensor([[0.4658, 0.8763, 0.6194, 0.4661, 0.2927, 0.1507, 0.0000],
        [0.4751, 0.9884, 0.5287, 0.6472, 0.1520, 0.0269, 0.0000],
        [0.5359, 0.9785, 0.7454, 0.5369, 0.0750, 0.1778, 0.0000],
        [0.5114, 0.9264, 0.8327, 0.6452, 0.2898, 0.0396, 0.0000]],
       dtype=torch.float64)
submitting candidates at time 2021-08-19T11:40:07-05:00
Model creation time: 0.3175 s
Candidate generation time: 26.35 s
Candidate(s): tensor([[0.5199, 0.8965, 0.5287, 0.5830, 0.2593, 0.0000, 0.0643],
        [0.4429, 0.9768, 0.6545, 0.5876, 0.2708, 0.0000, 0.0648],
        [0.4665, 0.8231, 0.6355, 0.6923, 0.2289, 0.0160, 0.0000],
        [0.4841, 0.9676

In [7]:
# create generator object
gen = MultiFidelityGenerator()

In [8]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

In [9]:
rec = gen.get_recommendation(model, X.vocs)
problem = AugmentedHartmann(negate=False)
problem(rec)

tensor([-3.1942])